# Fine-Tuning Nemotron-3 using Low-Rank Adaptation (LoRA)
Nemotron-3 is a robust, powerful family of Large Language Models that can provide compelling responses on a wide range of tasks. While the 8B parameter base model serves as a strong baseline for multiple downstream tasks, they can lack in domain-specific knowledge or proprietary or otherwise sensitive information. Fine-tuning is often used as a means to update a model for a specific task or tasks to better respond to domain-specific prompts. This notebook walks through preparing a dataset and using Low Rank Adaptation (LoRA) to fine-tune the base Nemotron-3 8B model from Hugging Face against the dataset.

The implementation of LoRA is based on the paper, [LoRA: Low-Rank Adaptation of Large Language Models](https://openreview.net/pdf?id=nZeVKeeFYf9) by Hu et al.

## Getting the model

A `Nemotron-3-8B-Base-4k.nemo` file should be present in `models/nemotron-3-8b-base-4k`. If not, run the following cell to download the model in NeMo format; this can take <ins>a few minutes</ins> to complete! This `Nemotron-3-8B-Base-4k.nemo` file will be used for fine-tuning for the remainder of the notebook.

In [ ]:
!bash /project/code/scripts/init.sh

## Preparing The Dataset
We will be using LoRA to teach our model to do Extractive Question Answering. The dataset being used for fine-tuning needs to be converted to a .jsonl file and follow a specific format. In general, question and answer datasets are easiest to work with by providing context (if applicable), a question, and the expected answer, though different downstream tasks work as well.

### Downloading the dataset
We will be using the [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".

In [ ]:
from datasets import load_dataset
from omegaconf import OmegaConf
import os
import wget
import sys
os.environ['OPENBLAS_NUM_THREADS'] = '8'

BRANCH='main'
DATA_DIR = "/project/data"
NEMO_DIR = "/project/code/notebooks"

os.makedirs(DATA_DIR, exist_ok=True)
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)

In [ ]:
# Download the SQuAD dataset
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!mv train-v1.1.json {SQUAD_DIR}
!mv dev-v1.1.json {SQUAD_DIR}

### Preprocessing the dataset
Datasets often need some form of preprocessing to convert it into a form ready for fine-tuning. LoRA (and all PEFT tuning) models expect at least two fields in the jsonl files. The `input` field should contain all the tokens necessary for the model to generate the `output`. For example for extractive QA, the `input` should contain the context text as well as the question.

```
[
    {"input": "User: Context: [CONTEXT_1] Question: [QUESTION_1]\n\nAssistant:", "output": [ANSWER_1]},
    {"input": "User: Context: [CONTEXT_2] Question: [QUESTION_2]\n\nAssistant:", "output": [ANSWER_2]},
    {"input": "User: Context: [CONTEXT_3] Question: [QUESTION_3]\n\nAssistant:", "output": [ANSWER_3]},
]
```
Note that we use keywords in the input like `Context:`, `Question:` to separate the text representing the context and question. We also use the keyword `User:` and end each of the input with `\n\nAssistant:` tokens. These are recommended because NeMo's instruction-tuned models are trained with a prefix of `User:` and suffix `\n\nAssistant:`.

The SQuAD dataset does not already reflect this, so let's go ahead and preprocess it to fit the above format. Feel free to take a look inside the `prompt_learning_squad_preprocessing.py` script.

In [ ]:
# Preprocess squad data
!python /opt/NeMo/scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py --sft-format --data-dir {SQUAD_DIR}

Let's split the datasets into train and validation files, and take a look at a few samples of the data to confirm the preprocessing is satisfactory. 

In [ ]:
# What the squad dataset looks like after processing
! head -5000 $SQUAD_DIR/squad_train.jsonl > $SQUAD_DIR/squad_short_train.jsonl
! head -500 $SQUAD_DIR/squad_val.jsonl > $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_train.jsonl

This format is used for both the `train` and `val` files. These files will be used as datasets for the fine-tuning process. The dataset is now ready to be used for fine-tuning the model!

## Configuring the job
With the dataset preparation finished, we need to update the default configuration for our fine-tuning job. The sample config file provided by NeMo is a good template to base our changes on. Let's load the file as an object that we can edit.

In [ ]:
config = OmegaConf.load("/opt/NeMo/examples/nlp/language_modeling/tuning/conf/megatron_gpt_finetuning_config.yaml")

With the config loaded, we can override certain settings for our environment. Many of the default values shown here would work but some key points are called out below:

* `config.trainer.precision=32` - This is the precision that will be used during fine-tuning. The model might be more accurate with higher values but it also uses more memory than lower precisions. If the fine-tuning process runs out of memory, try reducing the precision here.
* `config.trainer.devices=1` - This is the number of devices that will be used. If running on a multi-GPU system, increase this number as appropriate.
* `config.model.restore_from_path="/project/models/nemotron-3-8b-base-4k/Nemotron-3-8B-Base-4k.nemo"` - This is the path to the converted `.nemo` checkpoint from the beginning of the notebook. If the path changed, update it here.
* `config.model.data.train_ds.file_names` and `config.model.data.validation_ds.file_names` - If a different filename or path was used for the dataset files created earlier, specify the new values here.
* `config.model.global_batch_size` - If using a higher GPU count or if additional GPU memory allows, this value can be increased for higher performance. Note that higher batch sizes use more GPU memory.

In [ ]:
config.trainer.precision = 32
config.trainer.devices = 1
config.trainer.max_epochs = 4
config.trainer.val_check_interval = 1.0
config.trainer.max_steps=1000
config.trainer.val_check_interval=50

config.model.peft.peft_scheme="lora"  # we can also set this to adapter or ptuning or ia3
config.model.data.train_ds.file_names = [f"{SQUAD_DIR}/squad_short_train.jsonl"]
config.model.data.train_ds.concat_sampling_probabilities=[1.0]
config.model.data.validation_ds.file_names = [f"{SQUAD_DIR}/squad_short_val.jsonl"]
config.model.data.validation_ds.names=["squad_val"]
config.model.data.train_ds.prompt_template ="{input} {output}"

config.model.restore_from_path="/project/models/nemotron-3-8b-base-4k/Nemotron-3-8B-Base-4k.nemo"
config.exp_manager.exp_dir=f"{NEMO_DIR}/peft_lora"
config.exp_manager.explicit_log_dir="training_info"
config.model.micro_batch_size=1
config.model.global_batch_size=1
config.model.data.train_ds.num_workers=0  # 0 is recommended which just uses the main thread to process training examples
config.model.data.validation_ds.num_workers=0 # 0 is recommended which just uses the main thread to process the validation examples

os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

# Final model config
print(OmegaConf.to_yaml(config.model))

With the config settings updated, save it as a `.yaml` file that can be read by NeMo Toolkit during fine-tuning and save it to the fine-tuning configuration directory.

In [ ]:
# OmegaConf.save can also accept a `str` or `pathlib.Path` instance:
OmegaConf.save(config, "nemotron-config.yaml")

In [ ]:
!mv nemotron-config.yaml /opt/NeMo/examples/nlp/language_modeling/tuning/conf/

## Launching the job
With the model downloaded, the dataset prepped, and the config set, it is now time to launch the fine-tuning job! The following block launches the job on the specified number of GPUs. Depending on the size of the dataset and the GPU used, this could take anywhere from a few minutes to several hours to finish. As the model is tuned, checkpoints will be saved in the `training_info` directory inside the container. These checkpoints contain prompt embeddings which are used to send inference requests with the fine-tuned weights to deployed models so they respond as expected.

In [ ]:
!python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
    --config-name=nemotron-config.yaml

### Configuring the evaluation script
The configuration file for the evaluation job needs to be updated based on the provided template to reflect the changes for the experiment here. Load the template config and update some of the settings to match the local environment. Note the following settings may differ for custom datasets:
* `config.model.data.test_ds.file_names` - List any prediction files that should be used to evaluate the model. In general, it is recommended to have this be different from the training and validation files used during fine-tuning. For the sake of simplicity, we test on the validation set. You may generate your own test set if you wish. 
* `config.model.data.test_ds.names` - Change the name of the dataset used here.

In [ ]:
# Load the template config file
config = OmegaConf.load("/opt/NeMo/examples/nlp/language_modeling/tuning/conf/megatron_gpt_generate_config.yaml")

# Override required settings
config.model.restore_from_path="/project/models/nemotron-3-8b-base-4k/Nemotron-3-8B-Base-4k.nemo"
config.model.peft.restore_from_path="/project/code/notebooks/training_info/checkpoints/megatron_gpt_peft_lora_tuning.nemo"
config.model.data.test_ds.file_names=[f"{SQUAD_DIR}/squad_short_val.jsonl"]
config.model.data.test_ds.names=["squad"]
config.model.data.test_ds.global_batch_size=1
config.model.data.test_ds.micro_batch_size=1
config.model.data.test_ds.tokens_to_generate=30
config.model.data.test_ds.write_predictions_to_file=True
config.model.data.test_ds.output_file_path_prefix="/project/code/notebooks/predictions"
config.model.apply_rope_fusion=False
config.inference.greedy=True

# Save the new config file
OmegaConf.save(config, "nemotron-eval-config.yaml")

Once the config is saved, evaluation can be launched below. Depending on the size of the hardware and the number of inference examples, this may take a few minutes to complete. Results will be saved to `predictions_test_squad_inputs_preds_labels.jsonl`.

In [ ]:
!mv nemotron-eval-config.yaml /opt/NeMo/examples/nlp/language_modeling/tuning/conf/

In [ ]:
!python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py --config-name=nemotron-eval-config.yaml

Note depending on hardware and the number of examples used, the evaluation script may take a while to run since we are using a training container setting and not currently optimizing for inference. Once we are ready to serve the finetuned model for true deployment, we may then move the model to an optimized inference framework like Triton. 

After the evaluation script completes, view the results. Keep in mind the results you see may vary in quality: 
* The hyperparameters presented in this notebook are not optimal and only serve as examples. Could you be underfitting? Overfitting? These can be adjusted in the configs to improve performance! 

The point is fine tuning the out-of-the-box model to the general QA task is easy and straightforward with this workflow!

In [ ]:
!head -n 5 /project/code/notebooks/predictions_test_squad_inputs_preds_labels.jsonl 